# CH11 데이터 분석 (Pandas)

업로드된 **CH11 데이터 분석.ppt** 내용을 기반으로,  
`pandas` 데이터프레임 생성/접근/조작, 전처리(정렬·조건 열·결측치), 함수(describe/문자열/astype), 파일(CSV) 읽기·저장,  
그리고 **[플러스 예제] 자동차 배기량과 CO2 배출량 상관관계**까지  
👉 **설명 + 실습 예제**로 구성한 Jupyter Notebook입니다.

## 학습 목표
- Pandas **Series / DataFrame** 개념을 이해한다.
- DataFrame 생성, 열/행 접근, 연산, 조건 필터링을 할 수 있다.
- 전처리(열이름 변경, 정렬, 조건 열 추가, 빈도수, 결측치 처리, merge)를 수행한다.
- CSV 파일을 읽고(to/from) 저장할 수 있다.
- 산점도 및 상관계수로 관계를 분석할 수 있다.


## 0. 준비

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## 11.1 판다스 데이터프레임
Pandas는 데이터 처리/분석 라이브러리이며, 대표 구조는:
- **Series**: 1차원(인덱스+값)
- **DataFrame**: 2차원(행+열, 표 형태)

일반적으로 `import pandas as pd` 형태로 별칭을 사용합니다.

### 11.1-1) DataFrame 만들기: 딕셔너리로 생성

In [ ]:
df = pd.DataFrame({
    "name": ["민수", "영희", "철수", "지수"],
    "test": [20, 25, 15, 30],
    "assign1": [20, 15, 25, 30],
    "assign2": [30, 25, 35, 20],
})
df


### 11.1-2) 다른 방법: 리스트 + columns

In [ ]:
data = [
    ["민수", 20, 20, 30],
    ["영희", 25, 15, 25],
    ["철수", 15, 25, 35],
    ["지수", 30, 30, 20],
]
df2 = pd.DataFrame(data, columns=["name", "test", "assign1", "assign2"])
df2


### 11.1-3) 빈 DataFrame 만들고 열 추가

In [ ]:
df3 = pd.DataFrame()
df3["name"] = ["A", "B", "C"]
df3["score"] = [80, 90, 70]
df3


## 11.1-4) 데이터에 접근하기
### 열 추출
- `df['col']` : Series
- `df[['c1','c2']]` : DataFrame

In [ ]:
df["name"]


In [ ]:
df[["name", "test"]]


### pop으로 열 추출(추출한 열은 DataFrame에서 제거됨)

In [ ]:
df_pop = df.copy()
name_col = df_pop.pop("name")
print("추출된 name_col:")
print(name_col)
print("\n열이 제거된 df_pop:")
df_pop


### iterrows로 행 순회하기(학습용)
> 큰 데이터에서는 느릴 수 있어요. (학습/소량 데이터에 사용)

In [ ]:
for idx, row in df.iterrows():
    print(idx, row["name"], row["test"])


### 11.1-5) DataFrame 연산하기: 열 연산, 합계 열 만들기

In [ ]:
df_calc = df.copy()
df_calc["sum"] = df_calc["test"] + df_calc["assign1"] + df_calc["assign2"]
df_calc


### 조건 필터링: sum >= 80 인 행만

In [ ]:
df_calc[df_calc["sum"] >= 80]


### 조건 필터링 + 일부 열만 추출 (name, sum)

In [ ]:
df_calc.loc[df_calc["sum"] >= 80, ["name", "sum"]]


## 11.2 데이터 전처리
### 11.2-1) 열 이름 변경: rename

In [ ]:
df_r = df_calc.rename(columns={"test": "exam"})
df_r


### 11.2-2) 정렬: sort_values

In [ ]:
sorted_df = df_calc.sort_values(by="sum", ascending=False)
sorted_df


### 11.2-3) 조건에 따라 다른 값 부여: np.where로 grade 만들기

In [ ]:
df_g = df_calc.copy()
df_g["grade"] = np.where(df_g["sum"] >= 90, "A",
                 np.where(df_g["sum"] >= 80, "B", "C"))
df_g


### 조건에 따라 pass/fail 열 만들기

In [ ]:
df_g["result"] = np.where(df_g["sum"] >= 80, "pass", "fail")
df_g


### 11.2-4) 빈도수: value_counts

In [ ]:
df_g["result"].value_counts()


In [ ]:
df_g["grade"].value_counts()


### 11.2-5) 열 삭제: drop

In [ ]:
df_drop = df_g.drop(columns=["result"])
df_drop


### 11.2-6) DataFrame 시각화: 전체 성적(sum) 막대 그래프

In [ ]:
ind = np.arange(len(df_g))
plt.bar(ind, df_g["sum"])
plt.xticks(ind, df_g["name"])
plt.title("학생별 전체 성적(sum)")
plt.show()


### 누적 막대 그래프: test/assign1/assign2

In [ ]:
ind = np.arange(len(df_g))
plt.bar(ind, df_g["test"], label="test")
plt.bar(ind, df_g["assign1"], bottom=df_g["test"], label="assign1")
plt.bar(ind, df_g["assign2"], bottom=df_g["test"]+df_g["assign1"], label="assign2")
plt.xticks(ind, df_g["name"])
plt.title("학생별 시험/과제 점수(누적)")
plt.legend()
plt.show()


## 11.2-7) 결측치(NaN) 처리
- `isna()` : 결측치 여부
- `dropna()` : 결측치 제거
- `fillna()` : 결측치 대체

In [ ]:
df_na = pd.DataFrame({
    "name": ["민수", "영희", None, "지수"],
    "gender": ["M", None, "F", "F"],
    "score": [80, np.nan, 70, 90]
})
df_na


In [ ]:
df_na.isna()


### 열별 결측치 개수 (isna + sum)

In [ ]:
df_na.isna().sum()


### 결측치가 있는 행 제거: dropna()

In [ ]:
df_na.dropna()


### 특정 열만 기준으로 결측치 제거: subset

In [ ]:
df_na.dropna(subset=["name", "score"])


### 결측치 0으로 대체: fillna

In [ ]:
df_na.fillna(0)


### 열마다 다른 값으로 대체: 딕셔너리로 fillna

In [ ]:
df_na.fillna({"gender": "etc", "score": 0})


### 평균으로 대체: score 평균

In [ ]:
df_filled = df_na.copy()
df_filled["score"] = df_filled["score"].fillna(df_filled["score"].mean())
df_filled


## 11.2-8) 데이터프레임 결합: merge
공통 열(키)을 기준으로 결합합니다.

In [ ]:
df1 = pd.DataFrame({
    "name": ["민수", "영희", "철수"],
    "gender": ["M", "F", "M"],
    "age": [17, 16, 17]
})

df2 = pd.DataFrame({
    "name": ["민수", "영희", "철수"],
    "department": ["수학", "영어", "과학"]
})

df3 = pd.merge(df1, df2, on="name")
df3


In [ ]:
df4 = pd.DataFrame({
    "department": ["수학", "영어", "과학"],
    "head_of_department": ["김교수", "이교수", "박교수"]
})

df5 = pd.merge(df3, df4, on="department")
df5


## 11.3 판다스 함수
### 11.3-1) 통계 함수: describe

In [ ]:
df_g.describe()


### 11.3-2) 문자열 함수: str
문자열 열에 `.str`을 붙이면 벡터화된 문자열 처리가 가능합니다.

In [ ]:
df_str = pd.DataFrame({"temp": ["23.5C", "19.2C", "31.0C"], "height": ["170cm", "165cm", "180cm"]})
df_str


In [ ]:
# 'C', 'cm' 제거 후 숫자만 추출(정수 부분)
df_str["temp_int"] = df_str["temp"].str.replace("C", "", regex=False).astype(float).astype(int)
df_str["height_int"] = df_str["height"].str.replace("cm", "", regex=False).astype(int)
df_str


### astype로 자료형 변환

In [ ]:
df_str.dtypes


In [ ]:
df_str["height_int"].astype("float").dtypes


## LAB) 올림픽 메달 수 누적 막대 그래프
4개국의 금/은/동 메달 수를 DataFrame으로 만들고 누적 막대 그래프를 그립니다.

In [ ]:
medal = pd.DataFrame({
    "country": ["Korea", "Japan", "USA", "China"],
    "gold": [9, 27, 39, 38],
    "silver": [3, 14, 41, 32],
    "bronze": [9, 17, 33, 18]
})
medal


In [ ]:
ind = np.arange(len(medal))
plt.bar(ind, medal["gold"], label="gold")
plt.bar(ind, medal["silver"], bottom=medal["gold"], label="silver")
plt.bar(ind, medal["bronze"], bottom=medal["gold"]+medal["silver"], label="bronze")
plt.xticks(ind, medal["country"])
plt.title("Olympic medals (stacked)")
plt.legend()
plt.show()


## 11.4 파일 데이터 이용 (CSV)
### CSV 저장: to_csv
### CSV 불러오기: read_csv

> 노트북 환경에서는 현재 폴더(`/mnt/data` 등)에 저장됩니다.

In [ ]:
# 예시: precipitation 데이터프레임 만들기
precip = pd.DataFrame({
    "year": [2017, 2018, 2019, 2020, 2021],
    "spring": [250, 260, 240, 230, 255],
    "summer": [900, 880, 920, 950, 910],
    "fall": [320, 300, 310, 280, 290],
    "winter": [180, 190, 170, 200, 185]
})
precip["sum"] = precip[["spring","summer","fall","winter"]].sum(axis=1)
precip


In [ ]:
# CSV로 저장
out_csv = "precipitation_out.csv"
precip.to_csv(out_csv, index=False)
out_csv


In [ ]:
# CSV 읽기
df_read = pd.read_csv("precipitation_out.csv")
df_read


### 한글 CSV 인코딩 이슈
`UnicodeDecodeError`가 나면 `encoding='EUC-KR'`를 시도해보세요.

In [ ]:
# 예시(실행은 파일이 있을 때):
# df_kor = pd.read_csv("학급당학생수.csv", encoding="EUC-KR")
# df_kor.head()


### head / tail / shape 로 큰 데이터 확인

In [ ]:
df_read.head(), df_read.tail(), df_read.shape


## 11.5 [플러스 예제] 자동차 배기량과 CO2 배출량 상관관계
자료에서는 공공데이터포털에서 받은 CSV(예: `car20240731.csv`)를 불러와
- 배기량이 0인 행 제거
- 연료(휘발유/경유)로 분리
- 산점도 시각화
- 상관계수(corrcoef) 계산
을 수행합니다.

아래 코드는 **파일이 있으면 그대로 실행**되며, 파일이 없으면 **샘플 데이터로 구조를 연습**합니다.

In [ ]:
import os

csv_name = "car20240731.csv"  # 파일명이 다르면 바꿔주세요.

if os.path.exists(csv_name):
    df_car = pd.read_csv(csv_name)
    print("loaded:", df_car.shape)
else:
    # 샘플 데이터(학습용): 실제 통계가 아니라 '구조 연습'용입니다.
    rng = np.random.default_rng(42)
    fuel = rng.choice(["휘발유", "경유"], size=300, p=[0.65, 0.35])
    displacement = rng.integers(800, 3500, size=300)
    co2 = displacement * 0.05 + rng.normal(0, 20, size=300) + np.where(fuel=="경유", 10, 0)
    df_car = pd.DataFrame({
        "연료": fuel,
        "배기량": displacement,
        "CO2배출량": co2.round(1)
    })
    print("sample:", df_car.shape)

df_car.head()


In [ ]:
# 전처리: 배기량 0 제거(실제 파일에 0이 있을 수 있음)
df_car2 = df_car[df_car["배기량"] != 0].copy()

# 연료별 분리
df_gas = df_car2[df_car2["연료"].astype(str).str.contains("휘발")]
df_diesel = df_car2[df_car2["연료"].astype(str).str.contains("경유")]

print("휘발유:", df_gas.shape, "경유:", df_diesel.shape)


In [ ]:
# 산점도(한 그래프에 함께)
plt.scatter(df_gas["배기량"], df_gas["CO2배출량"], label="Gasoline", alpha=0.6)
plt.scatter(df_diesel["배기량"], df_diesel["CO2배출량"], label="Diesel", alpha=0.6)
plt.xlabel("배기량")
plt.ylabel("CO2 배출량")
plt.title("배기량 vs CO2 배출량")
plt.legend()
plt.show()


In [ ]:
# 상관계수(피어슨)
r_all = np.corrcoef(df_car2["배기량"], df_car2["CO2배출량"])[0,1]
r_gas = np.corrcoef(df_gas["배기량"], df_gas["CO2배출량"])[0,1] if len(df_gas) > 1 else np.nan
r_die = np.corrcoef(df_diesel["배기량"], df_diesel["CO2배출량"])[0,1] if len(df_diesel) > 1 else np.nan

r_all, r_gas, r_die


## 정리 체크리스트
- [ ] DataFrame을 만들고 열/행에 접근할 수 있다.
- [ ] 연산으로 새 열을 만들고 조건 필터링을 할 수 있다.
- [ ] rename/sort_values/where/value_counts/drop을 사용할 수 있다.
- [ ] 결측치 처리(isna/dropna/fillna)를 할 수 있다.
- [ ] merge로 DataFrame 결합을 할 수 있다.
- [ ] describe, 문자열 처리(.str), astype을 사용할 수 있다.
- [ ] read_csv/to_csv로 파일 데이터를 다룰 수 있다.
- [ ] 산점도 + 상관계수로 관계를 분석할 수 있다.
